In [ ]:
%matplotlib inline

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from remapping import mom_remapping
import gsw
import hycom
from importlib import reload

In [ ]:
temp_url = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/temperature/netcdf/decav/1.00/woa13_decav_t00_01v2.nc'
salt_url = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/salinity/netcdf/decav/1.00/woa13_decav_s00_01v2.nc'

temp_url_025 = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/temperature/netcdf/A5B2/0.25/woa13_A5B2_t00_04v2.nc'
salt_url_025 = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/salinity/netcdf/A5B2/0.25/woa13_A5B2_s00_04v2.nc'

In [ ]:
temp = Dataset(temp_url, 'r')
salt = Dataset(salt_url, 'r')

In [ ]:
lat = temp.variables['lat'][:]
lon = temp.variables['lon'][:]
dep = temp.variables['depth'][:]

In [ ]:
lon_w = -25.5

t_sect = temp.variables['t_an'][0,:,:,lon==lon_w].squeeze()
s_sect = salt.variables['s_an'][0,:,:,lon==lon_w].squeeze()

In [ ]:
# empty columns are entirely masked
empty = np.sum(~t_sect.mask, axis=0) == 0
empty[169:] = True # mask above Greenland

In [ ]:
lat = lat[~empty]
t_sect = t_sect[:,~empty]
s_sect = s_sect[:,~empty]

In [ ]:
sa_sect = np.empty_like(s_sect)
ct_sect = np.empty_like(t_sect)
rho_sect = np.empty_like(s_sect)
rhop_sect = np.empty_like(rho_sect)

for i in range(s_sect.shape[1]):
    sa_sect[:,i] = gsw.SA_from_SP(s_sect[:,i], dep, lon_w, lat[i])
    ct_sect[:,i] = gsw.CT_from_t(sa_sect[:,i], t_sect[:,i], dep)
    rho_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], dep)
    rhop_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], 2000)

In [ ]:
sa_int = sa_sect
ct_int = ct_sect

In [ ]:
sa_lay = (sa_int[1:,:] + sa_int[:-1,:]) / 2
ct_lay = (ct_int[1:,:] + ct_int[:-1,:]) / 2

In [ ]:
# depths of all interfaces on which observations are present
gr_int = np.ma.array(np.tile(dep.reshape(-1, 1), (1, sa_int.shape[1])), mask=sa_int.mask)

# thicknesses of all layers between interfaces
gr_th  = np.diff(gr_int, axis=0)

# bottom interface at each column
topo = gr_int.max(axis=0)

In [ ]:
remap_cs = mom_remapping.Remapping_Cs()
remap_cs.remapping_scheme = 4 # PQM_IH4IH3
remap_cs.degree = 4

In [ ]:
def remap(h):
    """
    Remap from original climatological grid according to h
    """

    sa_remap = np.empty_like(h)
    ct_remap = np.empty_like(h)

    # remap by columns
    for i in range(h.shape[1]):
        # we need to make sure we deal with unmasking here,
        # otherwise we'll get the fill values for thickness
        # and salt/temp, which would be just a little weird
        sa_remap[:,i] = mom_remapping.remapping_core_h(gr_th[:,i].compressed(),
                                                       sa_lay[:,i].compressed(),
                                                       h[:,i], remap_cs)
        ct_remap[:,i] = mom_remapping.remapping_core_h(gr_th[:,i].compressed(),
                                                       ct_lay[:,i].compressed(),
                                                       h[:,i], remap_cs)
        
    return sa_remap, ct_remap

# Replicating the MOM6 HyCOM grid

As a point of comparison, we'll implement MOM6's 75-level HyCOM grid on our climatology section, which will hopefully let us see what's going wrong with it, and whether any of our ideas can help.

Above, we used the 50-level target densities from the global_ALE experiment. Now we'll use the 75-level targets from OM4_025. The depth coordinates are generated by the string `FNC1:2,4000,4.5,.01`.

In [ ]:
s2_75 = [1010, 1014.3034, 1017.8088, 1020.843, 1023.5566, 1025.813, 
    1027.0275, 1027.9114, 1028.6422, 1029.2795, 1029.852, 1030.3762, 
    1030.8626, 1031.3183, 1031.7486, 1032.1572, 1032.5471, 1032.9207, 
    1033.2798, 1033.6261, 1033.9608, 1034.2519, 1034.4817, 1034.6774, 
    1034.8508, 1035.0082, 1035.1533, 1035.2886, 1035.4159, 1035.5364, 
    1035.6511, 1035.7608, 1035.8661, 1035.9675, 1036.0645, 1036.1554, 
    1036.2411, 1036.3223, 1036.3998, 1036.4739, 1036.5451, 1036.6137, 
    1036.68, 1036.7441, 1036.8062, 1036.8526, 1036.8874, 1036.9164, 
    1036.9418, 1036.9647, 1036.9857, 1037.0052, 1037.0236, 1037.0409, 
    1037.0574, 1037.0738, 1037.0902, 1037.1066, 1037.123, 1037.1394, 
    1037.1558, 1037.1722, 1037.1887, 1037.206, 1037.2241, 1037.2435, 
    1037.2642, 1037.2866, 1037.3112, 1037.3389, 1037.3713, 1037.4118, 
    1037.475, 1037.6332, 1037.8104, 1038]

In [ ]:
def dz_f1(n, dz_min, total, power, precision):
    dz = np.empty(n)
    
    # initial profile
    for i in range(n):
        dz[i] = (i / (n - 1)) ** power
    
    # rescale to total depth and round to precision
    dz[:] = (total - n*dz_min) * (dz[:] / np.sum(dz))
    dz[:] = np.around(dz[:], decimals=precision)
    
    # adjust bottom
    dz[-1] += total - np.sum(dz[:] + dz_min)
    dz[-1] = np.around(dz[-1], decimals=precision)
    
    dz[:] += dz_min
    
    return dz

Calculate nominal grid spacings and adjust to local topography.

In [ ]:
dz_75 = dz_f1(75, 2, 4000, 4.5, 2)
z_75 = np.insert(dz_75.cumsum(), 0, 0)[:,np.newaxis]

z_75_full = np.tile(z_75, (1, sa_sect.shape[1]))

np.putmask(z_75_full, z_75_full > topo, topo)

Calculate maximum interface depths and maximum layer thicknesses, also from the `dz_f1()` generating function.

In [ ]:
max_int_depth = np.insert(dz_f1(75, 5, 8000, 1, 2).cumsum(), 0, 0)
max_lay_thick = dz_f1(75, 400, 31000, 0.1, 2)

Translate to layer thicknesses and remap our climatology.

In [ ]:
h = np.diff(z_75_full, axis=0)
#h = np.maximum(h, 1e-3)

sa_map, ct_map = remap(h)

Now we can actually compute the grid according to the `hycom1` algorithm. We return `z_new`, which are just interfaces moved according to their target densities, and `z_bnd` which includes the additional step of interface adjustment to preserve the mixed layer.

In [ ]:
reload(hycom)

In [ ]:
z_iso, z_bnd, z_bnd_s = \
    hycom.hycom(h, sa_map, ct_map, s2_75, dz_75, max_int_depth, max_lay_thick)

# calculate T/S on both new grids for plotting
sa_iso, ct_iso = remap(np.diff(z_iso, axis=0))
sa_bnd, ct_bnd = remap(np.diff(z_bnd, axis=0))
sa_bnd_s, ct_bnd_s = remap(np.diff(z_bnd_s, axis=0))

## HyCOM Grids

In the first plot, we're showing the HyCOM grid without any adjustment to the isopycnal positions. In the second plot, we show the grid after the three kinds of adjustment: any interface which is shallower than its nominal position is moved to this nominal position, but interfaces can't be deeper than a specified depth. Additionally, layers can't be thicker than a specified thickness.

In [ ]:
plt.figure(figsize=(10,6))

plt.pcolormesh(lat, z_iso, gsw.rho(sa_iso, ct_iso, 2000))
plt.gca().invert_yaxis()
plt.colorbar()
plt.title('target isopycnals')
plt.plot(lat, z_iso.T, 'k', linewidth=0.5);

In [ ]:
plt.figure(figsize=(10,6))

#plt.pcolormesh(lat, z_bnd, gsw.rho(sa_bnd, ct_bnd, 2000))
plt.pcolormesh(lat, z_bnd, sa_bnd)
plt.gca().invert_yaxis()
plt.colorbar()
plt.plot(lat, z_bnd.T, 'k', linewidth=0.5)
plt.title('original hycom interfaces over absolute salinity')
plt.ylabel('depth')
plt.xlabel('latitude')

#plt.xlim(left=55)
#plt.xlim(right=-50)
#plt.ylim(bottom=300)

This is an alternate result, using the salinity modification of target pressure. As we hoped, the Denmark Strait overflow looks to be slightly more isopycnal (this problem is significantly magnified in the actual model output).

In [ ]:
plt.figure(figsize=(10,6))

#plt.pcolormesh(lat, z_bnd, gsw.rho(sa_bnd, ct_bnd, 2000))
plt.pcolormesh(lat, z_bnd_s, sa_bnd_s)
plt.gca().invert_yaxis()
plt.colorbar()
plt.plot(lat, z_bnd_s.T, 'k', linewidth=0.5)
plt.title('salinity-modified hycom interfaces over absolute salinity')
plt.ylabel('depth')
plt.xlabel('latitude')

#plt.xlim(left=55)
#plt.xlim(right=-50)
#plt.ylim(bottom=300)

## Alternate Salinity Transition

Here we modify the transition depth, but not the nominal depth.

In [ ]:
plt.figure(figsize=(10,6))

#plt.pcolormesh(lat, z_bnd, gsw.rho(sa_bnd, ct_bnd, 2000))
plt.pcolormesh(lat, z_bnd_s, sa_bnd_s)
plt.gca().invert_yaxis()
plt.colorbar()
plt.plot(lat, z_bnd_s.T, 'k', linewidth=0.5)
plt.title('alternate transition hycom interfaces over absolute salinity')
plt.ylabel('depth')
plt.xlabel('latitude')

#plt.xlim(left=55)
#plt.xlim(right=-50)
#plt.ylim(bottom=300)

Let's zoom on the overflow at the Denmark Strait, where we can see the impact of the reduction of target pressure with salinity. We also look at a segment near the Antarctic coast, where the halocline dictates that we should remain in z coordinates fairly deep. Our salinity modification leaves the target positions quite intact here.

In [ ]:
plt.figure(figsize=(10,8))

ax = plt.subplot(221)
plt.pcolormesh(lat, z_bnd, sa_bnd)
ax.invert_yaxis()
plt.plot(lat, z_bnd.T, 'k', linewidth=0.5)
plt.ylabel('depth')
plt.title('regular transition')

plt.xlim(left=55)
plt.ylim(bottom=300)

ax = plt.subplot(222)
plt.pcolormesh(lat, z_bnd_s, sa_bnd_s)
ax.invert_yaxis()
ax.set_yticks([])
plt.plot(lat, z_bnd_s.T, 'k', linewidth=0.5)
plt.title('salinity-modified transition')

plt.xlim(left=55)
plt.ylim(bottom=300)

ax = plt.subplot(223)
plt.pcolormesh(lat, z_bnd, sa_bnd)
ax.invert_yaxis()
plt.plot(lat, z_bnd.T, 'k', linewidth=0.5)
plt.ylabel('depth')
plt.xlabel('latitude')

plt.xlim(right=-50)
plt.ylim(bottom=1000)

ax = plt.subplot(224)
plt.pcolormesh(lat, z_bnd_s, sa_bnd_s)
ax.invert_yaxis()
ax.set_yticks([])
plt.plot(lat, z_bnd_s.T, 'k', linewidth=0.5)
plt.xlabel('latitude')

plt.xlim(right=-50)
plt.ylim(bottom=1000)

plt.tight_layout()

## Alternate Transition

In [ ]:
plt.figure(figsize=(10,8))

ax = plt.subplot(221)
plt.pcolormesh(lat, z_bnd, sa_bnd)
ax.invert_yaxis()
plt.plot(lat, z_bnd.T, 'k', linewidth=0.5)
plt.ylabel('depth')
plt.title('regular transition')

plt.xlim(left=55)
plt.ylim(bottom=300)

ax = plt.subplot(222)
plt.pcolormesh(lat, z_bnd_s, sa_bnd_s)
ax.invert_yaxis()
ax.set_yticks([])
plt.plot(lat, z_bnd_s.T, 'k', linewidth=0.5)
plt.title('salinity-modified transition')

plt.xlim(left=55)
plt.ylim(bottom=300)

ax = plt.subplot(223)
plt.pcolormesh(lat, z_bnd, sa_bnd)
ax.invert_yaxis()
plt.plot(lat, z_bnd.T, 'k', linewidth=0.5)
plt.ylabel('depth')
plt.xlabel('latitude')

plt.xlim(right=-50)
plt.ylim(bottom=1000)

ax = plt.subplot(224)
plt.pcolormesh(lat, z_bnd_s, sa_bnd_s)
ax.invert_yaxis()
ax.set_yticks([])
plt.plot(lat, z_bnd_s.T, 'k', linewidth=0.5)
plt.xlabel('latitude')

plt.xlim(right=-50)
plt.ylim(bottom=1000)

plt.tight_layout()